# Plotting complex numbers and complex function

This module exposes the `complex_plot` function which can be used to create different kinds of plot:
1. Plotting lists of complex numbers.
2. Plotting over a real range the real and imaginary part of a complex function.
3. Plotting over a real range the absolute value of a complex function colored by its argument.
4. Domain coloring of a complex function over a complex domain.
5. 3D plot of the absolute value of a complex function colored by its argument.

In the following tutorial we are not going to explore them all. Instead, we will look at the differences between the backends.

In [ ]:
from sympy import *
from spb import *
from spb.backends.plotly import PB
from spb.backends.k3d import KB
from spb.backends.bokeh import BB
var("x:z")
help(complex_plot)

## Real/Imaginary or Absolute/Argument

Let's say we'd like to plot the real and imaginary parts of a complex function over a real range. All we have to do is the following:

In [ ]:
expr = cos(x) + sqrt(I * x)
expr

In [ ]:
complex_plot(expr, (x, -2, 2), backend=PB)

We set `absarg=True` in order to create a plot of the magnitude of a complex function colored by its argument:

In [ ]:
complex_plot(expr, (x, -2, 2), absarg=True, backend=PB)

Note that Plotly is unable to plot gradient lines, so the change in phase is represented by colored markers. By hovering the markers we get a tooltip telling us useful information.

If we are only interested in the absolute value without argument-coloring, we can set `use_cm=False`:

In [ ]:
complex_plot(expr, (x, -2, 2), absarg=True, use_cm=False, backend=PB)

## Domain Coloring

[Domain coloring](https://en.wikipedia.org/wiki/Domain_coloring) is a technique for visualizing complex functions by assigning a color to each point of the complex plane. This module uses [cplot](https://github.com/nschloe/cplot) to accomplish this task. By quoting `cplot` documentation, we can control the coloring with two parameters:

> * `alpha = 1  # >= 0`: alpha can be used to adjust the use of colors. A value less than 1 adds more color which can help isolating the roots and poles (which are still black and white, respectively). alpha=0 ignores the magnitude of f(z) completely.
> * `colorspace = "cam16"  # "cielab", "oklab", "hsl"`: colorspace can be set to hsl to get the common fully saturated, vibrant colors. This is usually a bad idea since it creates artifacts which are not related with the underlying data. From Wikipedia: *Since the HSL color space is not perceptually uniform, one can see streaks of perceived brightness at yellow, cyan, and magenta (even though their absolute values are the same as red, green, and blue) and a halo around L = 1/2. Use of the Lab color space corrects this, making the images more accurate, but also makes them more drab/pastel.*


In [ ]:
complex_plot(gamma(z), (z, -4 - 2 * I, 4 + 2 * I), backend=PB)

By default, domain coloring plots automatically set `aspect="equal"`. 

From `cplot` documentation:

> The representation is chosen such that:
>
> * values around 0 are black,
> * values around infinity are white,
> * values around +1 are green,
> * values around -1 are deep purple,
> * values around +i are blue,
> * values around -i are orange.
>
> With this, it is easy to see where a function has very small and very large values, and the multiplicty of zeros and poles is instantly identified by counting the color wheel passes around a black or white point.

Also note that Plotly has a _bug_: the vertical axis is reversed, with negative values on the top and positive values on the bottom. We will get back to it later!

More generally, we can think of the result of domain coloring as a picture. The complex domain is discretized with `n1` points in the horizontal direction and `n2` points in the vertical direction. Therefore, the picture will have `(n1 - 1) (n2 - 1)` pixels. We can set `n1` and `n2` to refine the result, however Plotly will become slower and slower in rendering the results. In such cases, it is better to use a different backend, as we will later see.

Let's try to change the `alpha` and `colorspace` parameters:

In [ ]:
complex_plot(gamma(z), (z, -4 - 2 * I, 4 + 2 * I), backend=PB, alpha=0)

In [ ]:
complex_plot(gamma(z), (z, -4 - 2 * I, 4 + 2 * I), backend=PB, alpha=0, colorspace="hsl")

We can also plot the absolute value of the complex function colored by its argument in 3D:

In [ ]:
complex_plot(gamma(z), (z, -4 - 2 * I, 4 + 2 * I), 
             backend=PB, threed=True, zlim=(0, 10))

As you can see, Plotly by default is not keeping a fixed aspect ratio. Let's try the same with K3D:

In [ ]:
complex_plot(gamma(z), (z, -4 - 2 * I, 4 + 2 * I), 
             backend=KB, threed=True, zlim=(0, 10))

Considering that complex functions can go to infinity, a fixed-aspect ratio plotting library (like K3D) might not be the best choice!

## Using different backends

We have seen that Plotly is not really a good choice if we need to plot gradient lines. Let's try Bokeh:

In [ ]:
complex_plot(expr, (x, -2, 2), absarg=True, backend=BB)

Again, we can hover over the lines to get detailed information.

Let's now go back to the 2D domain coloring. To fix the reversed y-axis we need to use a different backend. Let's try with Bokeh:

In [ ]:
complex_plot(gamma(z), (z, -4 - 2 * I, 4 + 2 * I), backend=BB)

## Interactive plots

We can also use `iplot` to create interactive complex plots. We must remember to set `is_complex=True`.

Note: at this point in development, only line plots and 3D plots are supported.

In [ ]:
iplot(
    (z * sin(x * z) * exp(2 * pi * I / (y * z)), (z, -5, 5)),
    params = {
        x: (1, (0, 3)),
        y: (1, (-5, 5)),
    },
    fig_kw = dict(
        is_complex = True,
        backend = BB,
        absarg = True,
        n1 = 2000
    )
)

The user can change `absarg=False` and rerun the plot.

Let's now try with a 3D plot:

In [ ]:
iplot(
    ((z**2 + 1) / (x * (z**2 - 1)), (z, -4 - 2 * I, 4 + 2 * I)),
    params = {
        x: (1, (-2, 2))
    },
    fig_kw = dict(
        backend = KB,
        threed = True,
        is_complex = True,
        zlim = (0, 6)
    )
)